In [19]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
import google.generativeai as genai
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

In [49]:
GOOGLE_API_KEY = "***************************************"

In [50]:
genai.configure(api_key=GOOGLE_API_KEY)

In [7]:
def get_pdf_text(docs):
    text = ""
    for doc in docs:
        reader = PdfReader(doc)
        for page in reader.pages:
            text += page.extract_text()
    return text

In [20]:
def get_text_chunks(text):
    splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    chunks = splitter.split_text(text)
    return chunks

In [1]:
def get_vector_store(chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key="***************************************")
    vector_store = FAISS.afrom_texts(texts=chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")

In [34]:
import nest_asyncio
import asyncio

In [ ]:
nest_asyncio.apply()

def get_vector_store(chunks):
    embeddings = GoogleGenerativeAIEmbeddings(
        model="models/embedding-001",
        google_api_key="***************************************"
    )

    async def create_store():
        # Await the coroutine to resolve it properly
        vector_store = await FAISS.afrom_texts(texts=chunks, embedding=embeddings)
        vector_store.save_local("faiss_index")  # Save the index locally
        return vector_store

    # Run the coroutine directly in the existing event loop
    return asyncio.ensure_future(create_store())

In [3]:
def get_conversational_chain():
    prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide answer from provided context only,
    if answer is not available, just say "Answer not available", don't provide wrong answers.
    Context:\n {context}?\n
    Question:\n {question}\n

    Answer:\n\n
    """
    model = ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0.2, api_key="***************************************")
    prompt = PromptTemplate(template=prompt_template, input_variables=["context","question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)
    return chain

In [4]:
def user_input(user_question):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001", google_api_key="***************************************")
    
    new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
    docs = new_db.similarity_search(user_question)

    chain = get_conversational_chain()

    
    response = chain(
        {"input_documents":docs, "question": user_question}
        , return_only_outputs=True)

    print(response["output_text"])

In [46]:
pdf_docs = ["7181-attention-is-all-you-need.pdf","neural models in nlp.pdf"]

In [47]:
raw_text = get_pdf_text(pdf_docs)
text_chunks = get_text_chunks(raw_text)
get_vector_store(text_chunks)

<Task pending name='Task-2' coro=<get_vector_store.<locals>.create_store() running at C:\Users\Atharva Raut\AppData\Local\Temp\ipykernel_26320\2112239331.py:9>>

In [48]:
user_input("Explain First Derivative Saliency")

First-derivative saliency is a strategy inspired by back-propagation in vision.  It measures how much each input unit contributes to the final decision by approximating the class score (Sc(e)) with a linear function of e using the first-order Taylor expansion: Sc(e) ≈ w(e)Te + b, where w(e) is the derivative of Sc with respect to the embedding e.  The magnitude (absolute value) of the derivative indicates the sensitivity of the final decision to a change in a particular dimension, showing how much that dimension contributes to the final decision.  The saliency score is then given by S(e) = |w(e)|.

